In [9]:
#Importing the necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


zsh:1: command not found: nvcc


In [15]:
# Load the dataset
df = pd.read_csv('creditcard.csv')
X = df.drop('Class', axis=1).values
y = df['Class'].values
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_val, y_val = X[train_split:], y[train_split:]

#Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

In [23]:
#Handle unbalenced dataset
# class_counts = np.bincount(y_train)
# class_weights = 1. / class_counts
# sample_weights = class_weights[y_train]
# sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

#TensorDataset: Cette classe est utilisée pour créer un dataset a partir de tensor et itérer dessus.
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

#DataLoader: Cette classe est utilisée pour charger les données en lots (batches) pendant l'entraînement du modèle.
train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1000)



1000


- ```class_counts = np.bincount(y_train)``` : Compte le nombre d'occurence dans chaque classe (bincount car il n'y a que deux classe)
- ```class_weights = 1. / class_counts``` : Calcule les poids pour chaque classe en prenant l'inverse du nombre d'occurrences de chaque classe 
- ```sample_weights = class_weights[y_train]``` : Crée un tableau de poids pour chaque échantillon en fonction de sa classe 
- ```sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)```
- ```train_loader = DataLoader(train_dataset, batch_size=1000, sampler=sampler)```: Le DataLoader en PyTorch est responsable de la création des batches d'échantillons pour l'entraînement



In [21]:
for batch in train_loader:
    _, labels = batch
    print('class counts', np.bincount(labels.numpy()))


class counts [998   2]
class counts [1000]
class counts [1000]
class counts [1000]
class counts [999   1]
class counts [1000]
class counts [978  22]
class counts [1000]
class counts [992   8]
class counts [995   5]
class counts [990  10]
class counts [996   4]
class counts [996   4]
class counts [1000]
class counts [995   5]
class counts [988  12]
class counts [997   3]
class counts [995   5]
class counts [996   4]
class counts [1000]
class counts [999   1]
class counts [1000]
class counts [1000]
class counts [998   2]
class counts [1000]
class counts [1000]
class counts [999   1]
class counts [996   4]
class counts [1000]
class counts [999   1]
class counts [993   7]
class counts [999   1]
class counts [1000]
class counts [999   1]
class counts [1000]
class counts [1000]
class counts [1000]
class counts [1000]
class counts [1000]
class counts [999   1]
class counts [998   2]
class counts [997   3]
class counts [980  20]
class counts [993   7]
class counts [995   5]
class counts [998  

- DataLoader :
DataLoader est un autre utilitaire de PyTorch qui permet de charger des datasets de manière efficace en créant des minibatches d'échantillons. Il gère également le mélange (shuffling) et l'échantillonnage des données.
- train_loader :
train_loader est un DataLoader pour le dataset d'entraînement (train_dataset).
- batch_size=32 :
 Les données sont chargées en minibatches de 32 échantillons.
- sampler=sampler : 
Utilise un WeightedRandomSampler pour équilibrer les classes déséquilibrées lors de la sélection des échantillons pour chaque batch.
Cela permet d'augmenter la probabilité que les classes minoritaires soient bien représentées dans chaque batch, améliorant ainsi l'apprentissage du modèle sur des datasets déséquilibrés.
- val_loader :
val_loader est un DataLoader pour le dataset de validation (val_dataset).
- batch_size=32 : Les données sont chargées en minibatches de 32 échantillons.
- shuffle=True : Les échantillons sont mélangés (shuffled) aléatoirement avant d'être divisés en batches.
Le mélange des données est généralement utilisé pour le dataset de validation pour éviter que l'ordre des données n'affecte les performances du modèle.


In [17]:
class ClassifierCardModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ClassifierCardModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = len(np.unique(y))

model = ClassifierCardModel(input_dim, hidden_dim, output_dim)


In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# for inputs, labels in train_loader: 
#     print(inputs, labels)
#     print(inputs.shape, labels.shape)

Epoch 1/20, Train Loss: 4.434514903539502, Val Loss: 4.515296961131849, Accuracy: 99.86833327481479, F1 Train: 0.0016839741790625876, F1 Val: 0.9980254328108966
Epoch 2/20, Train Loss: 3.000717293878675, Val Loss: 2.324770638817235, Accuracy: 99.86833327481479, F1 Train: 0.009799554565701559, F1 Val: 0.9980254328108966
Epoch 3/20, Train Loss: 12.215949920153147, Val Loss: 14.40355906570167, Accuracy: 99.86833327481479, F1 Train: 0.0014114326040931546, F1 Val: 0.9980254328108966
Epoch 4/20, Train Loss: 4.9940619443526915, Val Loss: 2.1874930649472955, Accuracy: 99.86833327481479, F1 Train: 0.0, F1 Val: 0.9980254328108966
Epoch 5/20, Train Loss: 4.455466319124516, Val Loss: 3.5555000137864496, Accuracy: 99.86833327481479, F1 Train: 0.004457652303120356, F1 Val: 0.9980254328108966
Epoch 6/20, Train Loss: 9.273927323333657, Val Loss: 32.658645395647014, Accuracy: 99.86833327481479, F1 Train: 0.01995841995841996, F1 Val: 0.9980254328108966
Epoch 7/20, Train Loss: 14.421635030981218, Val Los